# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Подготовка данных

### Инициализируем сессию и прочитаем данные 

In [1]:
import pandas as pd 
import pandas.plotting
from pandas.plotting import scatter_matrix

import numpy as np
import six

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml import Pipeline

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

In [2]:
RANDOM_SEED = 2022

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Housing - Linear regression") \
                    .getOrCreate()

df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)
df.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [3]:
#создадим второй датафрейм без категориального признака
df_2 = df.select('longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value')
df_2.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [4]:
# выводим базовые статистики
df.describe().toPandas() 

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Данные были собраны в рамках переписи населения в США. Каждая строка содержит агрегированную статистику о жилом массиве. Жилой массив — минимальная географическая единица с населением от 600 до 3000 человек в зависимости от штата. Одна строка в данных содержит статистику в среднем о 1425.5 обитателях жилого массива.

В колонках датасета содержатся следующие данные:

- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

В большинстве колонок хранятся количественные данные, кроме одной — `ocean_proximity`. Она хранит категориальные значения.

### Проверим пропуски

In [5]:
#пропущенные значения в каждой колонке
columns = df.columns
lst = [None, np.nan, 'NULL']
for column in columns:
    check_col = F.col(column).cast(DoubleType())
    print(column, df.filter(check_col.isin(lst)).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Отсутствует примерно 0.5% данных. Заполним пропуски медианой 

In [6]:

df = df.na.fill(value=int(df.approxQuantile('total_bedrooms', [0.5], 0)[0]),subset=["total_bedrooms"])
df_2 = df_2.na.fill(value=int(df_2.approxQuantile('total_bedrooms', [0.5], 0)[0]),subset=["total_bedrooms"])

In [7]:
#разделим колонки на два типа: числовые и текстовые, которые представляют категориальные данные. Также выделим целевой признакcategorical_cols = ['ocean_proximity']
categorical_cols = ["ocean_proximity"]
numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
target = "median_house_value"

### Корреляции

In [8]:
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to target for ", i, df.stat.corr(target,i))

Correlation to target for  longitude -0.045966615117981745
Correlation to target for  latitude -0.14416027687465752
Correlation to target for  housing_median_age 0.10562341249321067
Correlation to target for  total_rooms 0.13415311380656275
Correlation to target for  total_bedrooms 0.049456861920854314
Correlation to target for  population -0.024649678888894876
Correlation to target for  households 0.06584265057005637
Correlation to target for  median_income 0.6880752079585578
Correlation to target for  median_house_value 1.0


### Разделение на выборки



In [9]:
#разделим датасет на две части — выборку для обучения и выборку для тестирования качества модели
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)

train_data_2, test_data_2 = df_2.randomSplit([.8,.2], seed=RANDOM_SEED)

## Обучение моделей



### Поcтроим  pipeline для обработки всех данных



In [10]:
# Build Pipeline 

categorical_cols = ["ocean_proximity"]
numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
target = "median_house_value"

stages = []

stringIndexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols], handleInvalid = 'keep')

encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])


categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
stages += [stringIndexer, encoder, categorical_assembler]

numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                                                            outputCol="numerical_features")

standardScaler = StandardScaler(inputCol='numerical_features',
                                                                outputCol="numerical_features_scaled")

all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
stages += [numerical_assembler,standardScaler,final_assembler]

# Run Data Through Pipeline
pipeline = Pipeline().setStages(stages)
pipeline_model = pipeline.fit(train_data)
prepped_train_data = pipeline_model.transform(train_data)
prepped_test_data = pipeline_model.transform(test_data)

In [11]:
pipeline.getStages()

[StringIndexer_3986978d6dd9,
 OneHotEncoder_ee67c8394a73,
 VectorAssembler_5e502093f9e1,
 VectorAssembler_f634e06ce305,
 StandardScaler_5ee94aabdf88,
 VectorAssembler_b00e2e1999c5]

### Обучим модель со всеми данными

In [12]:
lr = LinearRegression(labelCol=target, featuresCol='features')

model = lr.fit(prepped_train_data)

23/05/18 06:49:34 WARN Instrumentation: [ab8bff25] regParam is zero, which might cause numerical instability and overfitting.
23/05/18 06:49:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/18 06:49:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/05/18 06:49:35 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/05/18 06:49:35 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
23/05/18 06:49:35 WARN Instrumentation: [ab8bff25] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
23/05/18 06:49:37 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
23/05/18 06:49:37 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed


### Оценка метрик на трейн 

In [13]:
trainingSummary = model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 68839.094737
r2: 0.642508
MAE: 49832.725951


In [14]:
#посмотрим коэффициенты в регрессии
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [12420.586336481727,-27781.23152779337,18489.344466861992,8900.623976003111,-53112.07212806045,-53737.6592142435,12971.660891070753,-10754.706129980594,31259.229822912344,-43274.798998362785,27640.6138319465,73029.17414604359]
Intercept: -2249286.985353045


### Предсказания на тесте и оценка метрик

In [15]:
#предскажем тестовые значения
predictions = model.transform(prepped_test_data)
#посмотрим предсказания
predictions.select("prediction",target).show(5)



+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
| 152990.3715510266|          103600.0|
|214967.38530553272|           50800.0|
|142714.15593254054|           58100.0|
| 132483.0438585477|           68400.0|
|164098.83915469842|           72200.0|
+------------------+------------------+
only showing top 5 rows



In [16]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol=target,metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions))

test_result = model.evaluate(prepped_test_data)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result.meanAbsoluteError)



R Squared (R2) on test data = 0.653558
Root Mean Squared Error (RMSE) on test data = 68486.7
Mean Absolute Error (MAE) on test data = 49867


### Обучим модель только с числовыми данными

In [17]:
# Build Pipeline 

numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
target = "median_house_value"

stages = []


numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                                                            outputCol="numerical_features")

standardScaler = StandardScaler(inputCol='numerical_features',
                                                                outputCol="features")

 
stages += [numerical_assembler,standardScaler]

# Run Data Through Pipeline
pipeline = Pipeline().setStages(stages)
pipeline_model = pipeline.fit(train_data_2)
prepped_train_data_2 = pipeline_model.transform(train_data_2)
prepped_test_data_2 = pipeline_model.transform(test_data_2)

In [18]:
lr = LinearRegression(labelCol=target, featuresCol='features')

model = lr.fit(prepped_train_data_2)

23/05/18 06:49:42 WARN Instrumentation: [fe82e430] regParam is zero, which might cause numerical instability and overfitting.


### Оценка метрик на трейн

In [19]:
trainingSummary = model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 69780.095032
r2: 0.632668
MAE: 50944.509002


In [20]:
#посмотрим коэффициенты в регрессии
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [-86231.2606798842,-91412.62024754361,14006.3951769044,-14818.31139597535,35402.83692654434,-44083.3660836648,28238.633812066055,74878.58040390344]
Intercept: -3600459.0405836347


### Предсказания на тесте и оценка метрик

In [21]:
#предскажем тестовые значения
predictions = model.transform(prepped_test_data_2)
#посмотрим предсказания
predictions.select("prediction",target).show(5)


+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
|101397.02412080672|          103600.0|
|183325.89393649017|           50800.0|
|109609.16753835836|           58100.0|
| 80433.62265060423|           68400.0|
| 129998.2662690985|           72200.0|
+------------------+------------------+
only showing top 5 rows



In [22]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol=target,metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions))

test_result = model.evaluate(prepped_test_data_2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result.meanAbsoluteError)

R Squared (R2) on test data = 0.646173
Root Mean Squared Error (RMSE) on test data = 69212.9
Mean Absolute Error (MAE) on test data = 50866.6


In [23]:
spark.stop()

## Анализ результатов

В данной работе были проанализированы данные о жилье в Калифорнии в 1990 году. Были удалены пропуски, данные были предобработаны, проверены, закодированы и приведены к необходимому формату для дальнейшей работы. 

Далее были построены модели линейной регрессии на двух выборках: на полных данных и только числовых данных. Были получены следующие результаты:

На тренировочной выборке для полных данных:
RMSE: 68839.094737
r2: 0.642508
MAE: 49832.725951

На тренировочной выборке для числовых данных:
RMSE: 69780.095032
r2: 0.632668
MAE: 50944.509002



На тестовой выборке для полных данных:
R Squared (R2) on test data = 0.653558
Root Mean Squared Error (RMSE) on test data = 68486.7
Mean Absolute Error (MAE) on test data = 49867


На тестовой выборке для числовых данных:
R Squared (R2) on test data = 0.646173
Root Mean Squared Error (RMSE) on test data = 69212.9
Mean Absolute Error (MAE) on test data = 50866.6


|  | R2 | MAE | RMSE |
| --- | --- | --- | --- |
| train полная | 0.642 | 49832 | 68839 |
| train числовая | 0.632 | 50944 | 69780 |
| test полная | 0.653 | 49867 | 68486 |
| test числовая | 0.646 | 50866 | 69212 |

Коэффициент детерминации: единица минус отношение средней квадратичной ошибки модели к средней квадратичной ошибке среднего значения тестовой выборки. Собственно единица минус отношение (RMSE)2 к дисперсии реальных значений выборки. R2 показывает, что ближе к реальным данным: наши предсказания или просто среднее по выборке. В нашем случае метрика лежит от 0 до 1, что означает, что RMSE меньше, чем просто среднее по выборке. Значит, модель имеет право на существование.


MAE сообщает нам среднюю абсолютную разницу между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже MAE, тем лучше модель соответствует набору данных.

RMSE - квадратный корень из средней квадратичной разницы между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже RMSE, тем лучше модель соответствует набору данных.

В данном случае RMSE важнее, потому что он более чувствителен к наблюдениям, которые далеки от среднего, потому что стоимость разнится довольно сильно, это можно наблюдать в описательной табличке в начале анализа. Стандартное отклонение в датасете 115395. Также видно, что для всех моделей видна разница между MAE и RMSE.

Таким образом линейная регрессия показала лучшие результаты для полной выборки, включащую в себя категориальные признаки. Более того, метрики на тесте не сильно отличаются от метрик на трейне, что говорит о том, что модель не переобучена и хорошо показывает себя на новых данных.